In [ ]:
URL_RUTS = "https://www.dropbox.com/scl/fi/4f319idvh2eaesj1g62o6/df_inicial.csv?rlkey=jlppn9g5i2g2uzg59i7zloxpw&st=6ioi218o&dl=0"
UR_EXCEL = "https://www.dropbox.com/scl/fi/yxpqzs6gb2pdzlsvcwmtw/archivo1_final_corregido.xlsx?rlkey=8ckefjsr5w017ll0t19u8k4nw&st=g21oqb4x&dl=0"


In [3]:
import sys
import os
import subprocess
import glob

VENV_NAME = "mi_entorno"

if not os.path.exists(VENV_NAME):
    print(f"🔨 Creando entorno virtual '{VENV_NAME}'...")
    subprocess.check_call([sys.executable, "-m", "venv", VENV_NAME])
else:
    print(f"✅ Entorno virtual '{VENV_NAME}' ya existe.")

venv_pip = os.path.join(VENV_NAME, "bin", "pip")

print("⬇️ Instalando openpyxl en el entorno virtual...")
try:
    subprocess.check_call([venv_pip, "install", "openpyxl", "-q"])
    print("Librería instalada en la carpeta local.")
except Exception as e:
    print(f"Error instalando: {e}")
    raise e

site_packages_paths = glob.glob(f"{VENV_NAME}/lib/python*/site-packages")

if site_packages_paths:
    site_pkg = os.path.abspath(site_packages_paths[0])
    if site_pkg not in sys.path:
        print(f"💉 Inyectando librerías al sistema: {site_pkg}")
        sys.path.insert(0, site_pkg) 
    else:
        print("Librerías ya cargadas.")
else:
    raise FileNotFoundError("No se encontró la carpeta site-packages en el entorno virtual.")

import openpyxl
import pandas as pd
print(f"ÉXITO: openpyxl versión {openpyxl.__version__} cargado correctamente.")

🔨 Creando entorno virtual 'mi_entorno'...
⬇️ Instalando openpyxl en el entorno virtual...



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


✅ Librería instalada en la carpeta local.
💉 Inyectando librerías al sistema: /home/camilogt/mi_entorno/lib/python3.11/site-packages
📚 ÉXITO: openpyxl versión 3.1.5 cargado correctamente.


In [5]:

URL_RUTS = "https://www.dropbox.com/scl/fi/4f319idvh2eaesj1g62o6/df_inicial.csv?rlkey=jlppn9g5i2g2uzg59i7zloxpw&st=6ioi218o&dl=0"
UR_EXCEL = "https://www.dropbox.com/scl/fi/yxpqzs6gb2pdzlsvcwmtw/archivo1_final_corregido.xlsx?rlkey=8ckefjsr5w017ll0t19u8k4nw&st=g21oqb4x&dl=0"

FILE_CSV = "ruts.csv"
FILE_EXCEL_INPUT = "libro_indice_perquenco_RESULTADO.xlsx"
FILE_OUTPUT = "consolidado_final_con_datos_servel.xlsx"

if not os.path.exists(FILE_CSV):
    print("Descargando CSV gigante...")
    os.system(f'wget -O "{FILE_CSV}" "{URL_RUTS}" -q --show-progress')

if not os.path.exists(FILE_EXCEL_INPUT):
    print("Descargando Excel de entrada...")
    os.system(f'wget -O "{FILE_EXCEL_INPUT}" "{URL_EXCEL}" -q --show-progress')

print("Archivos listos.")

✅ Archivos listos.


In [6]:
print("Leyendo Excel de entrada...")
df_excel = pd.read_excel(FILE_EXCEL_INPUT)

print("Normalizando RUTs...")
# Limpieza: 12345678-9 -> 12345678
df_excel['rut_temp'] = df_excel['rut_encontrado'].astype(str).str.strip()
df_excel['rut_key'] = df_excel['rut_temp'].str.replace('.', '', regex=False).str.split('-').str[0]

print(f" Excel cargado: {len(df_excel)} filas.")
print(f" Ejemplo de llave: {df_excel['rut_key'].iloc[0]}")

📖 Leyendo Excel de entrada...
🧹 Normalizando RUTs...
✅ Excel cargado: 2614 filas.
   Ejemplo de llave: 10783056


In [7]:
print("Cargando Base Maestra (CSV)...")

try:
    df_ruts = pd.read_csv(
        FILE_CSV, 
        sep=';', 
        encoding='latin-1', 
        low_memory=False,
        usecols=lambda c: c != 'Unnamed: 0'
    )
    if len(df_ruts.columns) < 2:
        print("Reintentando con separador coma...")
        df_ruts = pd.read_csv(FILE_CSV, sep=',', encoding='latin-1', low_memory=False)

    print("CSV Cargado.")

except Exception as e:
    raise e

if 'RUN' not in df_ruts.columns:
    raise ValueError(f"Columna RUN no encontrada. Columnas: {list(df_ruts.columns)}")

df_ruts['RUN'] = df_ruts['RUN'].astype(str).str.strip().str.replace('.', '', regex=False)
df_ruts = df_ruts.drop_duplicates(subset=['RUN'], keep='last')
print(f"✅ Base lista: {len(df_ruts)} registros únicos.")

📖 Cargando Base Maestra (CSV)...
✅ CSV Cargado.
✅ Base lista: 14900189 registros únicos.


In [8]:
print("🔄 Ejecutando cruce...")

df_final = pd.merge(
    df_excel,
    df_ruts,
    left_on='rut_key',
    right_on='RUN',
    how='left'
)

for col in ['rut_key', 'rut_temp']:
    if col in df_final.columns:
        del df_final[col]

print(f"Guardando {FILE_OUTPUT}...")
df_final.to_excel(FILE_OUTPUT, index=False)
print("✨ ¡Proceso finalizado con éxito!")

🔄 Ejecutando cruce...
💾 Guardando consolidado_final_con_datos_servel.xlsx...
✨ ¡Proceso finalizado con éxito!


In [10]:
import sys
import subprocess
import os

try:
    import openpyxl
    import pandas as pd
    import numpy as np 
except ImportError:
    print("Instalando librerías necesarias...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", 
        "pandas", "openpyxl", "numpy", "--user", "--break-system-packages"
    ])
    import pandas as pd
    import numpy as np

NOMBRE_ARCHIVO_INPUT = "lip_consolidado.xlsx" 
NOMBRE_ARCHIVO_OUTPUT = "lip_consolidado_sufragio_corregido_02.xlsx"


print(f"Leyendo archivo: {NOMBRE_ARCHIVO_INPUT}...")

if not os.path.exists(NOMBRE_ARCHIVO_INPUT):
    print(f"❌ ERROR: No encuentro el archivo '{NOMBRE_ARCHIVO_INPUT}'.")
    sys.exit(1)

# Cargar el Excel
df = pd.read_excel(NOMBRE_ARCHIVO_INPUT)

# Verificar que la columna exista
if 'Sufragio' not in df.columns:
    print(f"❌ ERROR: La columna 'Sufragio' no existe en el Excel.")
    sys.exit(1)

print("🔄 Realizando reemplazo en columna 'Sufragio'...")

# --- CORRECCIÓN AQUÍ ---
# 1. Eliminamos .astype(str) para que los nulos sigan siendo nulos.
# 2. Usamos .str.replace directamente. Pandas ignorará las celdas vacías automáticamente.
df['Sufragio'] = df['Sufragio'].str.replace('Ã³', 'a', regex=False)

# 3. Limpieza de espacios (opcional pero recomendado)
df['Sufragio'] = df['Sufragio'].str.strip()

# 4. (SEGURIDAD EXTRA) Si por alguna razón quedaron textos "nan" literales
# (por ejemplo, si el excel original ya venía sucio), esto los vuelve a convertir en vacío real.
df['Sufragio'] = df['Sufragio'].replace({'nan': None, 'NaN': None})

# ---------------------------------------------------------
# 4. GUARDADO
# ---------------------------------------------------------
print(f"💾 Guardando resultado en: {NOMBRE_ARCHIVO_OUTPUT}...")
df.to_excel(NOMBRE_ARCHIVO_OUTPUT, index=False)

print("✅ ¡Listo! Las celdas vacías se mantuvieron vacías.")

📖 Leyendo archivo: lip_consolidado.xlsx...
🔄 Realizando reemplazo en columna 'Sufragio'...
💾 Guardando resultado en: lip_consolidado_sufragio_corregido_02.xlsx...
✅ ¡Listo! Las celdas vacías se mantuvieron vacías.
